# Step 3: SQL Data Loading & Preparation

**Purpose**: Load target data and prepare for Great Expectations validation

**Key Activities**:
- Load target table (DQ_LOGIC) data using existing connection
- Analyze table schema and data structure
- Profile data quality and completeness
- Prepare DataFrame for GX validation

**Expected Outputs**:
- Loaded DataFrame with target data
- Schema analysis and column information
- Data profiling summary
- Readiness status for step 4

**Note**: Assumes SQL Server connection is already established and tested.

In [0]:
# Run this in a separate cell before restarting
%pip install great_expectations sqlalchemy pyodbc pandas


  Using cached great_expectations-1.5.6-py3-none-any.whl.metadata (8.8 kB)
  Using cached sqlalchemy-2.0.41-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached altair-4.2.2-py3-none-any.whl.metadata (13 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.25.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached mistune-3.1.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached ruamel.yaml-0.18.14-py3-none-any.whl.metadata (24 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached greenlet-3.2.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (4.1 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached toolz-1.0.0-py3-none-any.whl.metadata (5.1

In [0]:
# =============================================================================
# UNIVERSAL DBFS CONTEXT SETUP
# Copy this cell to ALL notebooks that need GX context (Steps 2-8)
# =============================================================================


import os
import great_expectations as gx



DBFS_GX_ROOT = "/dbfs/FileStore/shared_uploads/great_expectations"
DBFS_DISPLAY_PATH = "/FileStore/shared_uploads/great_expectations"

print(f"DBFS Root: {DBFS_GX_ROOT}")
print(f"DBFS Display: {DBFS_DISPLAY_PATH}")

# Step 2: Create DBFS directory structure
print("CREATING GX STRUCTURE IN DBFS")
print("-" * 40)

try:
    os.makedirs(DBFS_GX_ROOT, exist_ok=True)
    
    subdirs = [
        "expectations",
        "checkpoints", 
        "data_docs",
        "validations",
        "profiling",
        "uncommitted"
    ]
    
    for subdir in subdirs:
        subdir_path = os.path.join(DBFS_GX_ROOT, subdir)
        os.makedirs(subdir_path, exist_ok=True)
        print(f"Created: {subdir}/")
    
    try:
        files = dbutils.fs.ls(DBFS_DISPLAY_PATH)
        print(f"DBFS verification successful: {len(files)} items found")
    except Exception as dbfs_error:
        print(f"DBFS verification warning: {dbfs_error}")
        
except Exception as e:
    print(f"Directory creation failed: {e}")
    raise
# Step 3: Initialize or get existing GX context
print("\nINITIALIZING GX CONTEXT")
print("-" * 40)

context = None
context_creation_methods = [
    ("DBFS FileDataContext", lambda: gx.get_context(project_root_dir=DBFS_GX_ROOT)),
    ("Ephemeral Context", lambda: gx.get_context(mode="ephemeral")),
    ("Default Context", lambda: gx.get_context())
]

for method_name, method_func in context_creation_methods:
    try:
        
        context = method_func()
       
        print(f"Context type: {type(context).__name__}")
        
        if method_name == "DBFS FileDataContext":
            print(f"Persistent storage: {DBFS_GX_ROOT}")
      
        
        break
        
    except Exception as e:
        print(f"{method_name} failed: {e}")
        continue

if context is None:
    raise RuntimeError("All context creation methods failed")

# Step 4: Context verification and configuration
print("\nCONTEXT VERIFICATION")
print("-" * 40)

try:
    datasources = context.list_datasources()
   
    
    key_methods = ['add_datasource', 'get_datasource', 'list_datasources']
    available_methods = [method for method in key_methods if hasattr(context, method)]
    print(f"Available methods: {len(available_methods)}/{len(key_methods)}")
    
    print("\nCONTEXT")
    print(f"Type: {type(context).__name__}")
   

    
except Exception as e:
    print(f"Context verification warning: {e}")

# Step 5: Export for use in other cells
print("\nEXPORTING CONTEXT")
print("-" * 40)

globals()['gx_context'] = context
globals()['DBFS_GX_ROOT'] = DBFS_GX_ROOT

print("Context exported as 'gx_context'")
print("DBFS path exported as 'DBFS_GX_ROOT'")





DBFS Root: /dbfs/FileStore/shared_uploads/great_expectations
DBFS Display: /FileStore/shared_uploads/great_expectations
CREATING GX STRUCTURE IN DBFS
----------------------------------------
Created: expectations/
Created: checkpoints/
Created: data_docs/
Created: validations/
Created: profiling/
Created: uncommitted/
DBFS verification successful: 6 items found

INITIALIZING GX CONTEXT
----------------------------------------
Context type: FileDataContext
Persistent storage: /dbfs/FileStore/shared_uploads/great_expectations

CONTEXT VERIFICATION
----------------------------------------
Available methods: 3/3

CONTEXT
Type: FileDataContext

EXPORTING CONTEXT
----------------------------------------
Context exported as 'gx_context'
DBFS path exported as 'DBFS_GX_ROOT'


In [0]:
# import json
# import pandas as pd
# from pyspark.sql import functions as F


# step3_results = {
#     "status": "running",
#     "data_loaded": False,
#     "target_table": "DQ_LOGIC",
#     "record_count": 0,
#     "columns": [],
#     "data_types": {},
#     "sample_data": [],
#     "ready_for_step4": False,
#     "error_message": None
# }


# # Use connection details from your working GX Helper v4
# try:
#     server = "shell-31-eun-sq-odduxhlfvttsvnrucqpy.database.windows.net"
#     database = "shell-31-eun-sqdb-edcruevdeslqjpdpwlcz"
    
#     # Use the CORRECT secret keys from your working code
#     username = dbutils.secrets.get("ADLSScope", "AzSQL-DBr-PROD")
#     password = dbutils.secrets.get("ADLSScope", "ADB-AzSQL-PROD-pwd")
    
#     # Build JDBC connection components for use in data loading
#     jdbc_url = f"jdbc:sqlserver://{server}:1433;database={database};user={username};password={password};encrypt=true;trustServerCertificate=true;loginTimeout=30;"
    
#     connection_properties = {
#         "user": username,
#         "password": password,
#         "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
#     }
    
#     print("✅ Using SQL Server connection configuration")
#     print(f"   • Server: {server}")
#     print(f"   • Database: {database}")
#     print(f"   • Username: {username}")
    
# except Exception as e:
#     print(f"❌ Failed to get connection details: {e}")
#     step3_results["error_message"] = f"Connection details failed: {e}"
#     step3_results["status"] = "error"

import json
import pandas as pd
from pyspark.sql import functions as F


step3_results = {
    "status": "running",
    "data_loaded": False,
    "target_table": "DQ_LOGIC",
    "record_count": 0,
    "columns": [],
    "data_types": {},
    "sample_data": [],
    "ready_for_step4": False,
    "error_message": None
}


try:
    server = "shell-31-eun-sq-odduxhlfvttsvnrucqpy.database.windows.net"
    port = 1433
    database = "shell-31-eun-sqdb-edcruevdeslqjpdpwlcz"
    

    client_id = dbutils.secrets.get(scope="ADLSScope", key="AzSQL-DBr-PROD")
    client_secret = dbutils.secrets.get(scope="ADLSScope", key="ADB-AzSQL-PROD-pwd")


    

    jdbc_url = f"""jdbc:sqlserver://{server}:{port};database={database};authentication=ActiveDirectoryServicePrincipal;encrypt=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;AADSecurePrincipalId={client_id};AADSecurePrincipalSecret={client_secret}"""
    
    connection_properties = {
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
        "authentication": "ActiveDirectoryServicePrincipal",
        "AADSecurePrincipalId": client_id,
        "AADSecurePrincipalSecret": client_secret,
        "encrypt": "true",
        "hostNameInCertificate": "*.database.windows.net",
        "loginTimeout": "30"
    }
    
  
    print(f"   • Server: {server}:{port}")
    print(f"   • Database: {database}")
    print(f"   • Authentication: Active Directory Service Principal")
    print(f"   • Client ID: {client_id[:8]}...")  # Show only first 8 chars for security
    
except Exception as e:
    print(f"Failed to get connection details: {e}")
    step3_results["error_message"] = f"Connection details failed: {e}"
    step3_results["status"] = "error"

   • Server: shell-31-eun-sq-odduxhlfvttsvnrucqpy.database.windows.net:1433
   • Database: shell-31-eun-sqdb-edcruevdeslqjpdpwlcz
   • Authentication: Active Directory Service Principal
   • Client ID: AMSDQDB_...


In [0]:
# =============================================================================
# TARGET TABLE DATA LOADING
# =============================================================================

if step3_results["status"] != "error":
    print(f"\n📊 TARGET TABLE DATA LOADING")
    print("-" * 50)
    
    target_table = "dbo.DQ_LOGIC"
    step3_results["target_table"] = target_table
    
    try:
        # Use the EXACT approach from your working GX Helper v4
        print(f"📥 Loading data from {target_table}...")
        
        # First, test connectivity with a count query (like your working v4)
        test_df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", "(SELECT COUNT(*) as test_count FROM dbo.DQ_LOGIC) as test") \
            .load()
        
        test_count = test_df.collect()[0]['test_count']
        print(f"✅ Connection successful! Found {test_count:,} rows in dbo.DQ_LOGIC")
        
        # Now load actual data using the working pattern
        data_df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", "(SELECT TOP 1000 * FROM dbo.DQ_LOGIC) as validation_data") \
            .load()
        
        # Convert to Pandas for easier GX integration (like your working v4)
        df = data_df.toPandas()
        
        # Get basic statistics
        record_count = len(df)
        columns = list(df.columns)
        
        step3_results["record_count"] = record_count
        step3_results["columns"] = columns
        step3_results["data_loaded"] = True
        
        print(f"✅ Data loaded successfully")
        print(f"📊 Records: {record_count}")
        print(f"📝 Columns: {len(columns)}")
        
        # Get column information from Pandas DataFrame
        print(f"\n📋 COLUMN INFORMATION:")
        
        # Get data types from the Pandas DataFrame
        for col_name in df.columns:
            col_type = str(df[col_name].dtype)
            step3_results["data_types"][col_name] = {
                "pandas_type": col_type,
                "has_nulls": df[col_name].isnull().any(),
                "null_count": df[col_name].isnull().sum()
            }
            print(f"  📌 {col_name}: {col_type}")
        
        print(f"📋 Columns: {list(df.columns)}")

    except Exception as e:
        print(f"❌ Data loading failed: {e}")
        step3_results["data_loaded"] = False
        step3_results["error_message"] = f"Data loading failed: {e}"
        step3_results["status"] = "error"


📊 TARGET TABLE DATA LOADING
--------------------------------------------------
📥 Loading data from dbo.DQ_LOGIC...
✅ Connection successful! Found 358,170 rows in dbo.DQ_LOGIC
✅ Data loaded successfully
📊 Records: 1000
📝 Columns: 9

📋 COLUMN INFORMATION:
  📌 id: int32
  📌 Global_Rule_Id: int32
  📌 HIERARCHY_ID: int32
  📌 DQ_SQL: object
  📌 ERROR_SQL: object
  📌 RECORD_CREATE_DATE: object
  📌 RECORD_CREATED_BY: object
  📌 RECORD_UPDATE_DATE: object
  📌 RECORD_UPDATED_BY: object
📋 Columns: ['id', 'Global_Rule_Id', 'HIERARCHY_ID', 'DQ_SQL', 'ERROR_SQL', 'RECORD_CREATE_DATE', 'RECORD_CREATED_BY', 'RECORD_UPDATE_DATE', 'RECORD_UPDATED_BY']


In [0]:
# =============================================================================
# TARGET TABLE DATA LOADING
# =============================================================================

if step3_results["status"] != "error":

    
    target_table = "dbo.DQ_LOGIC"
    step3_results["target_table"] = target_table
    
    try:

        
        # First, test connectivity with a count query (like your working v4)
        test_df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", "(SELECT COUNT(*) as test_count FROM dbo.DQ_LOGIC) as test") \
            .load()
        
        test_count = test_df.collect()[0]['test_count']
 

        data_df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", "(SELECT TOP 1000 * FROM dbo.DQ_LOGIC) as validation_data") \
            .load()
        

        df = data_df.toPandas()
        

        record_count = len(df)
        columns = list(df.columns)
        
        step3_results["record_count"] = record_count
        step3_results["columns"] = columns
        step3_results["data_loaded"] = True
        


        print(f"COLUMN INFORMATION:")
        

        for col_name in df.columns:
            col_type = str(df[col_name].dtype)
            step3_results["data_types"][col_name] = {
                "pandas_type": col_type,
                "has_nulls": df[col_name].isnull().any(),
                "null_count": df[col_name].isnull().sum()
            }
            print(f"{col_name}: {col_type}")
        
        print(f"Columns: {list(df.columns)}")

    except Exception as e:
        print(f"Data loading failed: {e}")
        step3_results["data_loaded"] = False
        step3_results["error_message"] = f"Data loading failed: {e}"
        step3_results["status"] = "error"

In [0]:
# =============================================================================
# DATA PROFILING & VALIDATION PREPARATION
# =============================================================================

if step3_results["data_loaded"]:
    print(f"\n📈 DATA PROFILING & VALIDATION PREPARATION")
    print("-" * 50)
    
    try:
        # Get sample data for inspection (df is now a Pandas DataFrame)
        sample_data = df.head(5).to_dict('records')
        step3_results["sample_data"] = sample_data
        
        print(f"📝 SAMPLE DATA (First 5 rows):")
        for i, row in enumerate(sample_data, 1):
            # Show first 3 columns from the dictionary
            first_three = dict(list(row.items())[:3])
            print(f"  Row {i}: {first_three}...")
        
        # Check for target columns mentioned in requirements
        target_columns = ["HIERARCHY_ID", "RECORD_CREATE_DATE"]
        missing_columns = []
        present_columns = []
        
        print(f"\n🎯 TARGET COLUMNS CHECK:")
        for col in target_columns:
            if col in columns:
                present_columns.append(col)
                print(f"  ✅ {col} - Present")
            else:
                missing_columns.append(col)
                print(f"  ❌ {col} - Missing")
        
        step3_results["target_columns"] = {
            "required": target_columns,
            "present": present_columns,
            "missing": missing_columns
        }
        
        # Basic data quality checks
        print(f"\n📊 BASIC DATA QUALITY CHECKS:")
        
        # Check for null values in key columns
        if present_columns:
            for col in present_columns:
                # Use Pandas methods since df is now a Pandas DataFrame
                null_count = int(df[col].isnull().sum())
                null_percentage = (null_count / record_count) * 100 if record_count > 0 else 0
                print(f"  📌 {col}: {null_count} nulls ({null_percentage:.1f}%)")
        
        # Check data freshness if RECORD_CREATE_DATE exists
        if "RECORD_CREATE_DATE" in present_columns:
            try:
                # Use Pandas methods for date analysis
                min_date = df["RECORD_CREATE_DATE"].min()
                max_date = df["RECORD_CREATE_DATE"].max()
                
                # Convert to strings for JSON serialization
                min_date_str = str(min_date)
                max_date_str = str(max_date)
                
                print(f"  📅 Date range: {min_date_str} to {max_date_str}")
                step3_results["date_range"] = {
                    "min_date": min_date_str,
                    "max_date": max_date_str
                }
            except Exception as e:
                print(f"  ⚠️  Could not analyze date range: {e}")
        
        # DataFrame is already Pandas, so no need to convert again
        step3_results["pandas_sample_size"] = len(df)
        print(f"📊 Pandas DataFrame available: {len(df)} rows")
        
        # Store DataFrame reference for next steps
        step3_results["dataframe_available"] = True
        
    except Exception as e:
        print(f"❌ Data profiling failed: {e}")
        step3_results["error_message"] = f"Data profiling failed: {e}"

# Final readiness assessment - ensure all values are Python native types
ready_for_step4 = bool(
    step3_results["data_loaded"] and
    step3_results["record_count"] > 0
)

step3_results["ready_for_step4"] = ready_for_step4
step3_results["status"] = "success" if ready_for_step4 else "error"

# Convert any remaining numpy types to Python native types
def convert_to_native_types(obj):
    """Convert numpy types to native Python types for JSON serialization"""
    import datetime
    import numpy as np
    
    if isinstance(obj, dict):
        return {k: convert_to_native_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native_types(v) for v in obj]
    elif isinstance(obj, (datetime.date, datetime.datetime)):
        return str(obj)  # Convert date/datetime objects to strings
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif hasattr(obj, 'dtype'):  # Other numpy types
        if 'bool' in str(obj.dtype):
            return bool(obj)
        elif 'int' in str(obj.dtype):
            return int(obj)
        elif 'float' in str(obj.dtype):
            return float(obj)
        else:
            return str(obj)
    else:
        return obj

# Clean the results dictionary
step3_results = convert_to_native_types(step3_results)

# Return results for orchestrator
dbutils.notebook.exit(json.dumps(step3_results))